# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kidal,18.4411,1.4078,22.02,13,0,4.50,ML,1737489153
1,1,talnakh,69.4865,88.3972,-41.87,94,50,1.00,RU,1737489154
2,2,balasore,21.4942,86.9317,17.71,54,0,1.53,IN,1737489155
3,3,namtsy,62.7161,129.6658,-19.78,100,100,0.37,RU,1737489157
4,4,morata de tajuna,40.2337,-3.4541,10.84,94,100,4.54,ES,1737489158


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Create a map plot for the city_data_df DataFrame, enabling a geogrpahic plot with color based on city names
city_map = city_data_df.hvplot.points(
    "Lng",                     
    "Lat",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500           
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Define ideal weather conditions
temp_condition = (city_data_df["Max Temp"].between(21, 27))  # More efficient than two comparisons
wind_condition = (city_data_df["Wind Speed"] < 4.5)
cloud_condition = (city_data_df["Cloudiness"] == 0)

# Create DataFrame with ideal weather conditions
ideal_cities_df = city_data_df[
    temp_condition &
    wind_condition & 
    cloud_condition
].copy()  # Using .copy() prevents SettingWithCopyWarning

# Drop any rows with null values and display info about remaining cities
ideal_cities_df = ideal_cities_df.dropna()
print(f"Found {len(ideal_cities_df)} cities with ideal weather conditions")
ideal_cities_df

Found 12 cities with ideal weather conditions


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,kodinar,20.7903,70.7031,21.02,57,0,3.24,IN,1737489177
24,24,alice springs,-23.7000,133.8833,22.75,21,0,0.00,AU,1737489182
136,136,hawaiian paradise park,19.5933,-154.9731,24.94,73,0,0.45,US,1737489316
147,147,mangrol,21.1167,70.1167,21.07,48,0,4.09,IN,1737489330
198,198,djibo,14.1022,-1.6306,25.27,10,0,3.77,BF,1737489395
399,399,tonala,20.6167,-103.2333,23.68,17,0,4.12,MX,1737489421
416,416,norman gardens,-23.3224,150.5393,23.29,70,0,0.00,AU,1737489672
435,435,marsabit,2.3284,37.9899,21.30,41,0,3.30,KE,1737489696
465,465,yeppoon,-23.1333,150.7333,24.05,90,0,2.49,AU,1737489732
504,504,harper,4.3750,-7.7169,26.66,78,0,2.87,LR,1737489781


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,kodinar,IN,20.7903,70.7031,57,
24,alice springs,AU,-23.7000,133.8833,21,
136,hawaiian paradise park,US,19.5933,-154.9731,73,
147,mangrol,IN,21.1167,70.1167,48,
198,djibo,BF,14.1022,-1.6306,10,
399,tonala,MX,20.6167,-103.2333,17,
416,norman gardens,AU,-23.3224,150.5393,70,
435,marsabit,KE,2.3284,37.9899,41,
465,yeppoon,AU,-23.1333,150.7333,90,
504,harper,LR,4.3750,-7.7169,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel, returning only the first hotel located 
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_adress = requests.get(base_url, params=params).json()

    # Check if "features" key exists and is non-empty
    if "features" in name_adress and len(name_adress["features"]) > 0:
        # Grab the hotel name if available
        hotel_df.loc[index, "Hotel Name"] = name_adress["features"][0]["properties"].get("name", "No name available")
    else:
        # No results found
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Print out the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kodinar - nearest hotel: Hotel
alice springs - nearest hotel: Aurora Alice Springs
hawaiian paradise park - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
djibo - nearest hotel: No name available
tonala - nearest hotel: Hotel Casa de las Palomas
norman gardens - nearest hotel: Kalka Hotel
marsabit - nearest hotel: Chicho Hotel
yeppoon - nearest hotel: Blossom's on Seaspray
harper - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
mukalla - nearest hotel: فندق الجابري


,City,Country,Lat,Lng,Humidity,Hotel Name
20,kodinar,IN,20.7903,70.7031,57,Hotel
24,alice springs,AU,-23.7000,133.8833,21,Aurora Alice Springs
136,hawaiian paradise park,US,19.5933,-154.9731,73,No hotel found
147,mangrol,IN,21.1167,70.1167,48,No hotel found
198,djibo,BF,14.1022,-1.6306,10,No name available
399,tonala,MX,20.6167,-103.2333,17,Hotel Casa de las Palomas
416,norman gardens,AU,-23.3224,150.5393,70,Kalka Hotel
435,marsabit,KE,2.3284,37.9899,41,Chicho Hotel
465,yeppoon,AU,-23.1333,150.7333,90,Blossom's on Seaspray
504,harper,LR,4.3750,-7.7169,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    "Lng",                     
    "Lat",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500, 
    hover_cols=["Hotel Name", "Country"]           
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)